In [2]:
import pandas as pd
import numpy as np

In [3]:
# Set fileName, and run all.
# logs.csv will contain the required data
#Updated code
fileName = 'btcusdt_1h.csv'

In [4]:
df = pd.read_csv(fileName)


In [5]:
def cmf(df, thresh=0):
  df_hour = df.copy()
  df_hour["datetime"] = pd.to_datetime(df_hour["datetime"])
  df_hour.set_index("datetime", inplace = True)

  df_hour.shape

  # Resampling to daily  frequency
  df_day = pd.DataFrame()
  df_day['open'] = df_hour.resample("D").open.first()
  df_day['close'] = df_hour.resample("D").close.last()
  df_day['high']=df_hour.resample("D").high.max()
  df_day['low'] = df_hour.resample("D").low.min()
  df_day['volume'] = df_hour.resample("D").volume.sum()
  df_day.reset_index(inplace=True)
  # print(df_day)

  df_hour.reset_index(inplace=True)
  signals = np.zeros(df_day.shape[0])
  MFV = df_day.volume*(2*df_day.close - df_day.low - df_day.high)/(df_day.high - df_day.low)
  res = MFV.rolling(20).sum()/df_day.volume.rolling(20).sum()
  pos = 0
  for i in range(signals.shape[0]-1):
      if res[i] > thresh and pos != 1:
          signals[i+1] = 1
          pos += 1
      elif res[i] < -thresh and pos != -1:
          signals[i+1] = -1
          pos -= 1
  df_day['signals'] = signals

  signals1 = np.zeros(df.shape[0])
  j = 0
  for i in range(df.shape[0]-1):
    if df_hour.datetime.iloc[i+1].day != df_hour.datetime.iloc[i].day:
      signals1[i] = df_day.signals[j]
      j+=1
  df['signals'] = signals1

  df['Index'] = range(0,len(df))
  # df['signals'] = signals
  df = df[['Index','datetime','open','high','low','close','volume','signals']]
  df.to_csv('logs.csv',encoding='utf-8',index=False)

In [6]:
cmf(df)